# Expresso Churn Prediction

## This Notebook is a classification algorithm that predicts whether an expresso customer(network provider) will churn or not churn

## Import the train Datasets

In [4]:
# The code was removed by Watson Studio for sharing.

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,dcf68cc2fb515ccad7d8b9b3bd80ee2a4b270063,SAINT-LOUIS,K > 24 month,17000.0,32.0,18000.0,6000.0,34.0,NaN,97.0,355.0,6.0,NaN,NaN,NO,62,All-net 500F=2000F;5d,35.0,0
1,71c44b5ba328db5c4192a80f7cf8f244d9350ed0,NaN,K > 24 month,4300.0,29.0,4427.0,1476.0,37.0,1764.0,8.0,3.0,0.0,NaN,2.0,NO,40,"Data: 100 F=40MB,24H",22.0,0
2,ce46411b1526c94f20a383b8cb188f8d27f82a0a,TAMBACOUNDA,K > 24 month,1500.0,3.0,1500.0,500.0,3.0,NaN,30.0,30.0,NaN,NaN,NaN,NO,32,All-net 500F=2000F;5d,3.0,0
3,f467cdb6669818373c26c2bad44e01ba66f97d21,FATICK,K > 24 month,1500.0,3.0,2497.0,832.0,4.0,0.0,159.0,45.0,19.0,NaN,NaN,NO,18,On net 200F=Unlimited _call24H,3.0,0
4,ec45e1a1888a32b5dcce0954cfec20c6e037db31,FATICK,K > 24 month,NaN,NaN,498.0,166.0,3.0,1.0,1.0,3.0,NaN,NaN,NaN,NO,50,NaN,NaN,0


## Data Wrangling

# Replacing Nans

In [5]:
import numpy as np
df['REGION'].replace(np.nan, 'DAKAR', inplace=True)

In [6]:
df['TOP_PACK'].replace(np.nan, 'All-net 500F=2000F;5d', inplace=True)

In [7]:
col=df.columns[3:14]
for i in col:
    mean=df[i].mean()
    df[i].replace(np.nan, mean, inplace=True)

In [8]:
mean=df['FREQ_TOP_PACK'].mean()
df['FREQ_TOP_PACK'].replace(np.nan, mean, inplace=True)

# Convert Categorical variables to Num

In [9]:
un_re=df['REGION'].unique()
num=range(14)
dic_region=dict()
for a,b in zip(un_re, num):
    dic_region[a]=b
    df['REGION'].replace(a, b, inplace=True)


In [10]:
un_te=df['TENURE'].unique()
num=range(8)
dic_tenure=dict()
for a,b in zip(un_te, num):
    dic_tenure[a]=b
    df['TENURE'].replace(a, b, inplace=True)

In [11]:
un_to=df['TOP_PACK'].unique()
num=range(111)
dic_top_pack=dict()
for a,b in zip(un_to, num):
    dic_top_pack[a]=b
    df['TOP_PACK'].replace(a, b, inplace=True)

In [12]:
df['MRG'].replace('NO', 0, inplace=True)

## Select feature sets

In [14]:
X=df[['REGION', 'TENURE', 'MONTANT', 'FREQUENCE_RECH', 'REVENUE',
       'ARPU_SEGMENT', 'FREQUENCE', 'DATA_VOLUME', 'ON_NET', 'ORANGE', 'TIGO',
       'ZONE1', 'ZONE2', 'MRG', 'REGULARITY', 'TOP_PACK', 'FREQ_TOP_PACK']]
X.shape

(400000, 17)

In [15]:
Y=df[['CHURN']]

## Import libraries from sklearn and scale datasets

In [16]:
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [46]:
scale=StandardScaler()
scale.fit(X)
XS=scale.transform(X)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  app.launch_new_instance()


## Train Test split and Model Training

In [47]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(XS, Y, test_size=0.2)

In [51]:
from sklearn.ensemble import GradientBoostingClassifier
clf=GradientBoostingClassifier()
clf.fit(x_train, y_train)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

## Test with log loss

In [53]:
y_pred=clf.predict_proba(x_test)
from sklearn.metrics import log_loss
log_loss(y_test,y_pred)

0.27698031671158435